In [1]:
# 스파크 세션 생성
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Spark study - 231223") \
    .getOrCreate()

131072x1 화면 크기가 잘못됐습니다. 문제가 예상됩니다
23/12/23 16:33:22 WARN Utils: Your hostname, KJH resolves to a loopback address: 127.0.1.1; using 172.22.50.175 instead (on interface eth0)
23/12/23 16:33:22 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/12/23 16:33:23 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
# 데이터프레임 생성
#df_genome_scores = spark.read.csv("file:///home/kjh/data/ml-latest/genome-scores.csv", header=True, inferSchema=True).createOrReplaceTempView('g_scores')
#df_genome_tags = spark.read.csv("file:///home/kjh/data/ml-latest/genome-tags.csv", header=True, inferSchema=True).createOrReplaceTempView('g_tags')
df_movies = spark.read.csv("file:///home/kjh/data/ml-latest/movies.csv", header=True, inferSchema=True)
df_ratings = spark.read.csv("file:///home/kjh/data/ml-latest/ratings.csv", header=True, inferSchema=True)
df_tags = spark.read.csv("file:///home/kjh/data/ml-latest/tags.csv", header=True, inferSchema=True)

In [3]:
df_movies.createOrReplaceTempView('movies')
df_ratings.createOrReplaceTempView('ratings')
df_tags.createOrReplaceTempView('tags')

In [4]:
df_user = spark.sql("""
                    SELECT r.userId, r.movieId, m.title, m.genres, r.rating, r.timestamp AS r_time, t.tag, t.timestamp AS t_time
                    FROM ratings r
                    INNER JOIN tags t
                    ON r.userId = t.userId 
                    AND r.movieId = t.movieId
                    INNER JOIN movies m
                    ON r.movieId = m.movieId
                    ORDER BY r.userId, r.movieId
                    """)

df_user.createOrReplaceTempView('user')
df_user.cache()

In [6]:
spark.sql("""
          SELECT *
          FROM user
          """).show()

+------+-------+--------------------+--------------------+------+----------+--------------------+----------+
|userId|movieId|               title|              genres|rating|    r_time|                 tag|    t_time|
+------+-------+--------------------+--------------------+------+----------+--------------------+----------+
|    10|    260|Star Wars: Episod...|Action|Adventure|...|   4.5|1430666645|        good vs evil|1430666558|
|    10|    260|Star Wars: Episod...|Action|Adventure|...|   4.5|1430666645|       Harrison Ford|1430666505|
|    10|    260|Star Wars: Episod...|Action|Adventure|...|   4.5|1430666645|              sci-fi|1430666538|
|    14|  58559|Dark Knight, The ...|Action|Crime|Dram...|   5.0|1311530004|         Atmospheric|1311530439|
|    14|  58559|Dark Knight, The ...|Action|Crime|Dram...|   5.0|1311530004|              Batman|1311530391|
|    14|  58559|Dark Knight, The ...|Action|Crime|Dram...|   5.0|1311530004|          comic book|1311530398|
|    14|  58559|Dar

In [8]:
spark.sql("""
          SELECT COUNT(*) cnt
          FROM user
          """).show()

+-------+
|    cnt|
+-------+
|1729292|
+-------+



In [9]:
spark.sql("""
          SELECT genres
          FROM user
          """).show(truncate=False)

+---------------------------+
|genres                     |
+---------------------------+
|Action|Adventure|Sci-Fi    |
|Action|Adventure|Sci-Fi    |
|Action|Adventure|Sci-Fi    |
|Action|Crime|Drama|IMAX    |
|Action|Crime|Drama|IMAX    |
|Action|Crime|Drama|IMAX    |
|Action|Crime|Drama|IMAX    |
|Action|Crime|Drama|IMAX    |
|Action|Crime|Drama|IMAX    |
|Action|Crime|Drama|IMAX    |
|Action|Crime|Drama|IMAX    |
|Action|Crime|Drama|IMAX    |
|Action|Crime|Drama|IMAX    |
|Action|Crime|Drama|IMAX    |
|Action|Crime|Drama|IMAX    |
|Action|Crime|Drama|IMAX    |
|Drama                      |
|Comedy|Crime|Drama|Thriller|
|Comedy|Crime|Drama|Thriller|
|Comedy|Crime|Drama|Thriller|
+---------------------------+
only showing top 20 rows



In [10]:
spark.sql("""
          DESCRIBE user
          """).show()

+--------+---------+-------+
|col_name|data_type|comment|
+--------+---------+-------+
|  userId|      int|   NULL|
| movieId|      int|   NULL|
|   title|   string|   NULL|
|  genres|   string|   NULL|
|  rating|   double|   NULL|
|  r_time|      int|   NULL|
|     tag|   string|   NULL|
|  t_time|   string|   NULL|
+--------+---------+-------+



In [15]:
spark.sql("""
          DESCRIBE ratings
          """).show()

+---------+---------+-------+
| col_name|data_type|comment|
+---------+---------+-------+
|   userId|      int|   NULL|
|  movieId|      int|   NULL|
|   rating|   double|   NULL|
|timestamp|      int|   NULL|
+---------+---------+-------+



In [20]:
spark.sql("""
          SELECT from_unixtime(timestamp) AS datetime
          FROM ratings
          """).show(5)

+-------------------+
|           datetime|
+-------------------+
|2008-11-04 02:52:19|
|2008-11-05 15:04:46|
|2008-11-04 02:31:43|
|2008-11-04 03:00:04|
|2008-11-04 02:58:39|
+-------------------+
only showing top 5 rows



In [22]:
df_user = spark.sql("""
                    SELECT userId, movieId, title,
                      explode(split(genres, '\\\\|')) AS genre,
                      rating, from_unixtime(r_time) AS r_time, tag, from_unixtime(t_time) AS t_time
                    FROM user
                    """)

df_user.createOrReplaceTempView('user')
df_user.cache()

In [33]:
spark.sql("""
          SELECT *
          FROM user
          """).show()

+------+-------+--------------------+---------+------+-------------------+-------------+-------------------+
|userId|movieId|               title|    genre|rating|             r_time|          tag|             t_time|
+------+-------+--------------------+---------+------+-------------------+-------------+-------------------+
|    10|    260|Star Wars: Episod...|   Action|   4.5|2015-05-04 00:24:05| good vs evil|2015-05-04 00:22:38|
|    10|    260|Star Wars: Episod...|Adventure|   4.5|2015-05-04 00:24:05| good vs evil|2015-05-04 00:22:38|
|    10|    260|Star Wars: Episod...|   Sci-Fi|   4.5|2015-05-04 00:24:05| good vs evil|2015-05-04 00:22:38|
|    10|    260|Star Wars: Episod...|   Action|   4.5|2015-05-04 00:24:05|Harrison Ford|2015-05-04 00:21:45|
|    10|    260|Star Wars: Episod...|Adventure|   4.5|2015-05-04 00:24:05|Harrison Ford|2015-05-04 00:21:45|
|    10|    260|Star Wars: Episod...|   Sci-Fi|   4.5|2015-05-04 00:24:05|Harrison Ford|2015-05-04 00:21:45|
|    10|    260|Sta

In [34]:
spark.sql("""
          SELECT COUNT(*) cnt
          FROM user
          """).show()

+-------+
|    cnt|
+-------+
|4683548|
+-------+



23/12/23 17:51:10 WARN StandaloneAppClient$ClientEndpoint: Connection to KJH.:7077 failed; waiting for master to reconnect...
23/12/23 17:51:10 WARN StandaloneSchedulerBackend: Disconnected from Spark cluster! Waiting for reconnection...
23/12/23 17:51:10 WARN StandaloneAppClient$ClientEndpoint: Connection to KJH.:7077 failed; waiting for master to reconnect...
23/12/23 17:51:15 ERROR TaskSchedulerImpl: Lost executor 3 on 172.22.50.175: Remote RPC client disassociated. Likely due to containers exceeding thresholds, or network issues. Check driver logs for WARN messages.
23/12/23 17:51:15 WARN BlockManagerMasterEndpoint: No more replicas available for rdd_271_2 !
23/12/23 17:51:15 WARN BlockManagerMasterEndpoint: No more replicas available for rdd_271_6 !
23/12/23 17:51:15 WARN BlockManagerMasterEndpoint: No more replicas available for rdd_271_12 !
23/12/23 17:51:15 WARN BlockManagerMasterEndpoint: No more replicas available for rdd_271_10 !
23/12/23 17:51:16 ERROR TaskSchedulerImpl: Lo